In [2]:
# Importing Important libraries
import pandas as pd
from tqdm import tqdm
import re
import time
import urllib
import warnings
from tqdm import trange
import sys
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [40]:
url="https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp=000000008"

In [41]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; apple64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

In [42]:
res=requests.get(url,headers=headers,verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'arc-sos.state.al.us'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [43]:
sou=BeautifulSoup(res.content,'lxml')

In [44]:
sou.text.strip()

"Business Entity Records | Alabama Secretary of State\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama Secretary of State\n\n\n\n\n\n\n\nMain navigation\n\n\nSOS Office\n\n\nBiography\n\n\nHistory\n\n\nVoter Registration\n\n\nEvent Scheduling\n\n\nFees & Funds\n\n\nFee Schedule\n\n\nFunds Guide\n\n\nVoter List Fee Schedule\n\n\n\n\nCommittees\n\n\nCommittees Guide\n\n\nMeeting Minutes\n\n\n\n\nStudent Center\n\n\nLegislative Internship\n\n\nSOS Internship Application\n\n\nStudent Poll Worker Internship\n\n\nKids Center\n\n\n\n\nQuarterly & Annual Reports\n\n\n\n\nElections\n\n\nServices\n\n\nAdministrative Services\n\n\nAuthentications\n\n\nTrademarks\n\n\nLands\n\n\n\n\nUCC Services\n\n\nUCC Home\n\n\nOnline Filings\n\n\nFiling Fees\n\n\nProcedures\n\n\nUCC Downloads\n\n\n\n\nBusiness Entities\n\n\nOnline Services\n\n\nDomestic Corporations\n\n\nForeign Corporations\n\n\nHomeowner's Associations\n\n\nLLCs\n\n\nLLPs\n\n\nBusiness Downloads\n\n\n\nAthlete Agents\n\n\nOath of Office Information Center\n

In [45]:
cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()

In [46]:
cmp_name

'A-One Body Shop of Florence, Inc.'

In [17]:
def harvest_page(num):
  loo_df=pd.DataFrame()
  url=f'https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp={num}'
  headers={ "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding":"gzip,deflate,br",
            "Accept-Language":"en-US,en;q=0.9,ta;q=0.8",
            "Connection":"keep-alive",
            "Cookie":"_ga=GA1.1.2085200574.1702026901;_ga_HEFYBZ2QN2=GS1.1.1702285595.5.1.1702285603.0.0.0",
            "Host":"arc-sos.state.al.us",
            "Sec-Fetch-Dest":"document",
            "Sec-Fetch-Mode":"navigate",
            "Sec-Fetch-Site":"none",
            "Sec-Fetch-User":"?1",
            "Upgrade-Insecure-Requests":"1",
            "User-Agent":"Mozilla/5.0(WindowsNT10.0;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/120.0.0.0Safari/537.36",
            "sec-ch-ua":"'Not_ABrand';v='8','Chromium';v='120','GoogleChrome';v='120'",
            "sec-ch-ua-mobile":"?0",
            "sec-ch-ua-platform":"Windows"
          }
  try:
    res=requests.get(url,headers=headers,timeout=40)
    sou=BeautifulSoup(res.content,'lxml')
    data_find=sou.text.strip()
    try:
        cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()
        stus_code=res.status_code
        datas0=[]
        datas1=[]

        for i in sou.find('div',attrs={'align':'center'}).find_all('tr'):
          try:
            attri_name=i.find('td',class_="aiSosDetailDesc").text.strip()
            detail_value=i.find('td',class_="aiSosDetailValue").text.strip()

            datas1.append(detail_value)
            datas0.append(attri_name)

          except:
            pass

        for i in range(len(datas0)):
          loo_df[datas0[i]]=[str(datas1[i])]

        loo_df['company']=[cmp_name]
        loo_df['Url_number']=[num]
        loo_df['Status']=[stus_code]

        loo_df['Test']=["Allow"]
    except:
       if data_find in "You have exceeded the number accesses allowed.":
        time.sleep(1)
        return harvest_page(num)
       else:
        loo_df['Test']=["Not Have"]

    return loo_df
  except:
    return loo_df

In [18]:
# dtas_oyut=harvest_page('000000002')
harvest_page('000000007')

,Test
0,Not Have


In [ ]:
# dtas_oyut.to_csv('ayeoirtqie.csv')

In [19]:
str_n="000000000"
range_v=[]
for i in range(1,1000):
  st_v=f"{len(str(i))}"
  num_f=str_n[int(st_v):]+str(i)
  range_v.append(num_f)

In [20]:
len(range_v)

999

In [ ]:
collect_frame=[]
def compile_run():
    with tqdm(total=len(range_v),desc="harvest") as progress:
        with ThreadPoolExecutor(max_workers=10) as executor:
            for i in executor.map(harvest_page,range_v):
               collect_frame.append(i)
               progress.update()
               time.sleep(.1)
               if len(collect_frame)%5==0:
                time.sleep(.2)
compile_run()

harvest:  69%|██████▉   | 694/999 [35:54<00:38,  7.83it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
final_df=pd.DataFrame()
for i in collect_frame:
    final_df=pd.concat([final_df,i],axis=0,ignore_index=True)

In [ ]:
final_df.groupby('Test').count()

,Entity ID Number,Entity Type,Principal Address,Principal Mailing Address,Status,Place of Formation,Formation Date,Registered Agent Name,Registered Office Street Address,Registered Office Mailing Address,...,Member Mailing Address,Withdrawn Date,Service of Process,Qualify Date,Filed Date,Place of Formation Changed,Cancelled Date,General Partner Name,General Partner Street Address,General Partner Mailing Address
Test,,,,,,,,,,,,,,,,,,,,,
Allow,458,458,458,458,458,458,458,458,456,456,...,158,2,2,15,1,1,1,1,1,1
Not Have,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#final_df.to_csv('/content/drive/MyDrive/Company Projects/Small task/Sample_Output_file.csv')

### Async Code

In [21]:
import aiohttp
import asyncio
import nest_asyncio

In [22]:
async def harvest_page(session,num):
        loo_df=pd.DataFrame()
        try:
          headers={ "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,application/signed-exchange;v=b3;q=0.7",
                        "Accept-Encoding":"gzip,deflate,br",
                        "Accept-Language":"en-US,en;q=0.9,ta;q=0.8",
                        "Connection":"keep-alive",
                        "Cookie":"_ga=GA1.1.2085200574.1702026901;_ga_HEFYBZ2QN2=GS1.1.1702285595.5.1.1702285603.0.0.0",
                        "Host":"arc-sos.state.al.us",
                        "Sec-Fetch-Dest":"document",
                        "Sec-Fetch-Mode":"navigate",
                        "Sec-Fetch-Site":"none",
                        "Sec-Fetch-User":"?1",
                        "Upgrade-Insecure-Requests":"1",
                        "User-Agent":"Mozilla/5.0(WindowsNT10.0;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/120.0.0.0Safari/537.36",
                        "sec-ch-ua":"'Not_ABrand';v='8','Chromium';v='120','GoogleChrome';v='120'",
                        "sec-ch-ua-mobile":"?0",
                        "sec-ch-ua-platform":"Windows"
                      }
          url=f'https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp={num}'
          async with session.get(url, headers=headers) as response:
              status_code = response.status
              page_content = await response.text()
              #res=requests.get(url,headers=headers,timeout=40)
              sou=BeautifulSoup(page_content,'lxml')
              data_find=sou.text.strip()
              try:
                  cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()
                  stus_code=res.status_code
                  datas0=[]
                  datas1=[]

                  for i in sou.find('div',attrs={'align':'center'}).find_all('tr'):
                    try:
                      attri_name=i.find('td',class_="aiSosDetailDesc").text.strip()
                      detail_value=i.find('td',class_="aiSosDetailValue").text.strip()

                      datas1.append(detail_value)
                      datas0.append(attri_name)

                    except:
                      pass

                  for i in range(len(datas0)):
                    loo_df[datas0[i]]=[str(datas1[i])]

                  loo_df['company']=[cmp_name]
                  loo_df['Url_number']=[num]
                  loo_df['Status']=[stus_code]

                  loo_df['Test']=["Allow"]
              except:
                if data_find in "You have exceeded the number accesses allowed.":
                  time.sleep(2)
                  return harvest_page(num)
                else:
                  loo_df['Url_number']=[num]
                  loo_df['Test']=["Not Have"]

              return loo_df
        except:
          loo_df['Url_number']=[num]
          loo_df['Test']=["Error"]
          return loo_df

In [23]:
collect_frame=[]
async def main(list_num):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for perma in list_num:
            tasks.append(harvest_page(session, perma))
        results = []

        with tqdm(total=len(list_num),desc="harvest") as progress:
          for task in asyncio.as_completed(tasks):
              data=await task
              collect_frame.append(data)
              time.sleep(.1)
              progress.update(1)
              if len(collect_frame)%5==0:
                time.sleep(.2)

if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main(range_v))

harvest: 100%|██████████| 999/999 [08:28<00:00,  1.97it/s]


In [24]:
final_df=pd.DataFrame()
for i in collect_frame:
    final_df=pd.concat([final_df,i],axis=0,ignore_index=True)

In [31]:
tests=final_df[['Url_number', 'Test']]

In [39]:
tests[tests['Test']!="Allow"]

,Url_number,Test
65,000000279,Error
67,000000275,Error
68,000000286,Error
70,000000567,Error
71,000000281,Error
...,...,...
994,000000829,Error
995,000000534,Error
996,000000530,Error
997,000000247,Error


In [27]:
len(collect_frame)

999